# Librairies

In [1]:
import requests
import json
import os
from faker import Faker
from tqdm import tqdm 
import os
import pathlib

#import session_info
#session_info.show()

# Create folders to store data 

In [2]:
current_path = pathlib.Path().resolve()

json_5K = os.path.join(current_path, "json1")
json_2_5K = os.path.join(current_path, "json2")
json_1_5K = os.path.join(current_path, "json3")

os.mkdir(json_5K)
os.mkdir(json_2_5K)
os.mkdir(json_1_5K)

You can generate the number of random address you want in a certain country thanks to Reverse Geocoder API on GCP (also avalaible on AWS): https://console.cloud.google.com/marketplace/product/geolytica-public/reverse-geocode-lite-3geonames-org?project=silent-blade-354311
Once you have launched the API, they will provide you your own IP to call the API (Feature: Site address)

# Generate Addresses

In [ ]:
faker_json = {"BE": "fr_BE", "EL": "el", "PT": "pt_PT", "UK": "uk", "ES": "es", "RO": "ro", "CZ": "cz", "FR": "fr", "HU": "hu", "HR": "hr", "RU": "ge", "SK": "sk", "DE": "de", "IT": "it", "NL": "nl", "FI": "fi", "PL": "pl", "LV": "lv"}

countries_5000 = ["BE", "LT", "PT", "BG", "UK", "ES", "LU", "RO", "CZ", "FR", "HU", "SI", "DK", "HR", "MT", "SK", "DE", "IT", "NL", "FI", "EE", "CY", "AT", "SE" , "IE", "LV", "PL", "NO", "CH", "LI", "IS", "RU"]
countries_2500 = ["AL", "ME", "RS", "MK", "TR", "BA", "AM", "BY", "GE", "AZ", "MD", "UA"]
countries_1500 = ["DZ", "LB", "SY", "EG", "LY", "TN", "IL", "MA", "JO", "PS"]

counter = 0
iteration = {1:5000, 2:2500, 3:1500} #We choose the number of iteration for each country batch

for countries in tqdm([countries_5000,countries_2500,countries_1500]):
    counter+=1
    for code in tqdm(countries): #We loop throught all the countries where we want to generate addresses
        addresses_array = []
        for sample in tqdm(range(0,iteration[counter])): #We decide how many addresses we want for each country 
            try:
                resp = requests.get("http://<YOUR_SITE_ADDRESS_IP>/randomland."+code+".json")
                data = resp.json()

                data_dict = {}
                data_dict["state"] = data["nearest"]["state"]
                data_dict["name"] = data["nearest"]["name"]
                data_dict["city"] = data["nearest"]["city"]
                data_dict["prov"] = data["nearest"]["prov"]
                data_dict["region"] = data["nearest"]["region"]
                data_dict["latt"] = data["nearest"]["latt"]
                data_dict["longt"] = data["nearest"]["longt"]

                try:
                    fake = Faker(faker_json[code])
                    data_dict["address"] = fake.address()
                except: #Faker doesn't support this address language. So, we put an english address
                    fake = Faker('en')
                    data_dict["address"] = fake.address()

                addresses_array.append(data_dict)

            except: #If a country isn't supported by the API, or another error occur, we catch the error to iterate 
                print("An API call went wrong, we iterate")

        json_country = json.dumps(addresses_array)
        with open('json'+str(counter)+'/data_'+code+'.json', 'w') as outfile:
            json.dump(json_country, outfile,indent=4)